In [1]:
import pandas as pd

In [2]:
from pandas_profiling import ProfileReport

In [3]:
from sklearn import preprocessing

In [4]:
df = pd.read_csv("flight_delays_data.csv", parse_dates=[8])

In [5]:
le = preprocessing.LabelEncoder()
df["Arrival_enc"] = le.fit_transform(df[["Arrival"]])
df["Airline_enc"] = le.fit_transform(df[["Airline"]])


/home/tim/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tim/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
df["flight_year"] = df["flight_date"].apply(lambda x: x.year)
df["flight_month"] = df["flight_date"].apply(lambda x: x.month)
df["flight_day"] = df["flight_date"].apply(lambda x: x.day)

df["delay_time"] = df["delay_time"].apply(lambda x: 9999 if x == "Cancelled" else float(x))

df["label"] = df["is_claim"].apply(lambda x: True if x != 0 else False)


In [7]:
df.head()

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,Arrival_enc,Airline_enc,flight_year,flight_month,flight_day,label
0,1582499,UO686,27,HKG,KIX,UO,10,0.4,2016-07-01,0,73,110,2016,7,1,False
1,1582501,CI7868,17,HKG,TNN,CI,11,0.5,2015-04-23,0,140,26,2015,4,23,False
2,1582504,PR301,14,HKG,MNL,PR,11,0.0,2014-04-08,0,94,85,2014,4,8,False
3,1582508,LD327,37,HKG,SIN,LD,3,0.1,2013-09-15,0,128,59,2013,9,15,False
4,1582509,KA5390,40,HKG,PEK,KA,9,0.5,2015-10-05,0,111,53,2015,10,5,False


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899114 entries, 0 to 899113
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   flight_id     899114 non-null  int64         
 1   flight_no     899114 non-null  object        
 2   Week          899114 non-null  int64         
 3   Departure     899114 non-null  object        
 4   Arrival       899114 non-null  object        
 5   Airline       897400 non-null  object        
 6   std_hour      899114 non-null  int64         
 7   delay_time    899114 non-null  float64       
 8   flight_date   899114 non-null  datetime64[ns]
 9   is_claim      899114 non-null  int64         
 10  Arrival_enc   899114 non-null  int64         
 11  Airline_enc   899114 non-null  int64         
 12  flight_year   899114 non-null  int64         
 13  flight_month  899114 non-null  int64         
 14  flight_day    899114 non-null  int64         
 15  label         899

In [13]:
label = df.pop("label")
df.pop("flight_id")
df.pop("flight_no")
df.pop("Departure")
df.pop("Arrival")
df.pop("Airline")
df.pop("is_claim")
data = df

In [9]:
corr = df.corr()

In [10]:
corr.style.background_gradient()

,flight_id,Week,std_hour,delay_time,is_claim,Arrival_enc,Airline_enc,flight_year,flight_month,flight_day,label
flight_id,1.000000,-0.000409,-0.000416,0.000650,-0.000041,0.001616,-0.001176,0.000569,-0.000473,0.000615,-0.000041
Week,-0.000409,1.000000,-0.000176,0.000285,-0.014874,-0.000057,0.001449,-0.428666,0.996447,0.096420,-0.014874
std_hour,-0.000416,-0.000176,1.000000,0.001303,-0.000481,-0.029533,0.019807,-0.003853,-0.000176,-0.000377,-0.000481
delay_time,0.000650,0.000285,0.001303,1.000000,0.801306,0.000751,0.023776,0.015390,0.001405,-0.010365,0.801306
is_claim,-0.000041,-0.014874,-0.000481,0.801306,1.000000,0.011480,0.017709,0.030985,-0.014816,0.001088,1.000000
Arrival_enc,0.001616,-0.000057,-0.029533,0.000751,0.011480,1.000000,-0.042319,-0.027319,-0.000035,0.000072,0.011480
Airline_enc,-0.001176,0.001449,0.019807,0.023776,0.017709,-0.042319,1.000000,-0.004888,0.001485,-0.000596,0.017709
flight_year,0.000569,-0.428666,-0.003853,0.015390,0.030985,-0.027319,-0.004888,1.000000,-0.431298,-0.003407,0.030985
flight_month,-0.000473,0.996447,-0.000176,0.001405,-0.014816,-0.000035,0.001485,-0.431298,1.000000,0.014613,-0.014816
flight_day,0.000615,0.096420,-0.000377,-0.010365,0.001088,0.000072,-0.000596,-0.003407,0.014613,1.000000,0.001088


In [12]:
from tpot import TPOTClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

/home/tim/.local/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/tim/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [18]:
xgb = XGBClassifier(n_estimators=100,
                    max_depth=6,
                    learning_rate=0.5,
                    n_jobs=6,
                    verbosity=0)
len_features = len(data.columns)
#pylint:disable=invalid-name
X_train, _, y_train, _ = train_test_split(data,
                                          label.values,
                                          train_size=0.75,
                                          test_size=0.25,
                                          random_state=39)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
/home/tim/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/tim/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/tim/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/tim/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is depre

KeyError: None

In [19]:
sfs = SFS(xgb,
          k_features=(1, len_features),
          forward=True,
          floating=True,
          verbose=1,
          scoring='neg_mean_absolute_error',
          cv=5,
          n_jobs=6)
sfs = sfs.fit(X_train, y_train)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
/home/tim/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/tim/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/tim/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/tim/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is depre

KeyError: None

In [17]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [14]:
%matplotlib inline

In [11]:
!pip install tpot mlxtend

     |████████████████████████████████| 87 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 157 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 173.6 MB 14 kB/s  eta 0:00:01     |██████████████                  | 75.4 MB 1.9 MB/s eta 0:00:52
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=89c90540ff0757d0c0c35208a8c1b9cac0650f5d8cb370f1e27da1288e6bc71c
  Stored in directory: /home/tim/.cache/pip/wheels/a8/bb/8f/6b9328d23c2dcedbfeb8498b9f650d55d463089e3b8fc0bfb2
Successfully built stopit
